In [2]:
import os
import sys
import re
import numpy as np
import pandas as pd
import scipy.io as sio
from scipy import signal
import numpy as np
from PyEMD import EMD
import torch
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime

/home/tseringj/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
folder_path = {"Long_words": "/home/tseringj/final_project/Long_Words",
               "Short_Long_words": "/home/tseringj/final_project/Short_Long_words",
               "Short_words": "/home/tseringj/final_project/Short_words",
               "Vowels": "/home/tseringj/final_project/Vowels"}

words_dict = {
    "Long_words": ["cooperate", "independent"],
    "Short_Long_words": ["cooperate", "in"],
    "Short_words": ["out", "in", "up"],
    "Vowels": ["a", "i", "u"]
}

numeric_labels = {
    "Long_words": {"cooperate": 0, "independent": 1},
    "Short_Long_words": {"cooperate": 0, "in": 1},
    "Short_words": {"out": 0, "in": 1, "up": 2},
    "Vowels": {"a": 0, "i": 1, "u": 2}
}

In [4]:
# datasub2 = sio.loadmat('Long_Words/sub_2b_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
#datasub3 = sio.loadmat('Long_Words/sub_3b_ch80_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
# datasub6 = sio.loadmat('Long_Words/sub_6_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
# datasub7 = sio.loadmat('Long_Words/sub_7_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
# datasub9 = sio.loadmat('Long_Words/sub_9c_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
# datasub11 = sio.loadmat('Long_Words/sub_11b_ch64_l_eog_removed_256Hz.mat')['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']

In [5]:

matrix_to_load = "eeg_data_wrt_task_rep_no_eog_256Hz_last_beep"

def load_EEG(type, subject_no):
    path = folder_path[type]
    words = words_dict[type]
    for subject_file in os.scandir(path):
        if not (subject_file.is_file() and subject_file.name.endswith('.mat') and
                int(re.search("[0-9]+", subject_file.name).group(0)) == subject_no):
            continue
        mat = sio.loadmat(subject_file.path)[matrix_to_load]
        
        temp = f"{path}/temp_files3"
        if not os.path.exists(temp):
            os.mkdir(temp)
        temp = f"{temp}/{subject_no}"

        if not os.path.exists(temp):
            os.mkdir(temp)
        X = []
        Y = []
        for index, eeg in np.ndenumerate(mat):
            temp2 = f"{temp}/{words[index[0]]}_{index[1] + 1}.npy" #storing each trial
            X.append(temp2)
            Y.append(words[index[0]])
            if not os.path.exists(temp2):
                np.save(temp2, eeg)
    return np.array(X), np.array(Y)

In [6]:
# # function for data augmentation

# def train_augmentation(X,Y):
#     final_X=np.empty((0,64,512))
#     label=np.empty((0,1))
    
#     for i in range(len(X)):
#         result=np.empty((4,64,512))
#         with open(X[i], 'rb') as f:
#             data = np.load(f)
#             #channels_to_select = [i for i in range(64) if i not in [0, 9, 32, 63]]  # Channels to select (excluding 0, 9, 32, and 63)
#             data=data[:,:1152]
#             # Loop through the data with a stride of 64 samples
#             k=0
#             for j in range(0, 1125, 192):
#                 if j+512 >= 1152:
#                     break
            

#                 result[k,:,:] = data[:, j:j+512]
#                 k+=1
                
#             final_X=np.vstack((final_X, result))
#             if numeric_labels[type][Y[i]]==0:
#               label=np.vstack((label, np.zeros((4,1))))
#             else:
#               label=np.vstack((label, np.ones((4,1))))
            
    
    
        
#     return final_X,label

In [7]:
def Calculate_PSD(signal):

    frequencies, psd = signal.welch(signal, fs=256)  # Adjust the sampling frequency (fs) if needed

    # Plot the PSD
    plt.figure(figsize=(8, 4))
    plt.plot(frequencies, psd)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power Spectral Density')
    plt.title('Power Spectral Density of IMF')
    plt.grid(True)
    plt.show()
    return

In [8]:
from scipy.stats import entropy

In [9]:


def compute_time_domain_features(eeg_data):
    
    time_features = {}

    # Mean
    time_features['mean'] = np.mean(eeg_data)

    # # Standard deviation
    time_features['std'] = np.std(eeg_data)

    # # Kurtosis
    #time_features['kurtosis'] = kurtosis(eeg_data)

    # # Energy
    #time_features['Energy'] = np.sum(np.square(eeg_data))

    # # RMS (Root Mean Square)
    time_features['RMS'] = np.sqrt(np.mean(np.square(eeg_data)))

    # # Zero-crossing rate
    # zero_crossings = np.where(np.diff(np.sign(eeg_data)))[0]
    # zero_crossing_rate = len(zero_crossings) / (len(eeg_data) - 1)
    # time_features['Zero-crossing rate'] = zero_crossing_rate

    # # Hjorth parameters
    # # Hjorth parameters
    #time_features['hjorth_activity'] = np.sqrt(np.sum(np.square(eeg_data)) / time_features['std']**2)
    #time_features['hjorth_mobility'] = time_features['mean'] / time_features['std']
    #time_features['hjorth_complexity'] = kurtosis(eeg_data) / (time_features['std']**4)

    # Skewness
    time_features['Skewness'] = skew(eeg_data)

    # Median
    time_features['Median'] = np.median(eeg_data)


    # # Range
    #time_features['Range'] = np.ptp(eeg_data)

    # # Inter-quartile range
    # time_features['Inter-quartile range'] = np.percentile(eeg_data, 75) - np.percentile(eeg_data, 25)

    # # Variance
    # time_features['Variance'] = np.var(eeg_data)

    # # Autocorrelation
    # autocorr = np.correlate(eeg_data, eeg_data, mode='full')
    # time_features['Autocorrelation'] = autocorr[len(autocorr)//2:]


    # Compute power spectral density using Welch's method
    # frequencies, psd = signal.welch(eeg_data, fs=256)

    # Calculate total power as a spectral feature
    # total_power = np.sum(psd)
    # time_features['total_power'] = total_power

    # Calculate spectral entropy as a spectral feature
    # normalized_psd = psd / np.sum(psd)  # Normalize PSD values
    # spectral_entropy = entropy(normalized_psd, base=2)
    # time_features['spectral_entropy'] = spectral_entropy

    

    # Assuming you have the IMF signal stored in a variable called 'imf_signal'



    return list(time_features.values())

In [10]:
def feature_extraction(data):
    row, col=data.shape
    feature=[]
    for i in range(row):
        emd = EMD()
        IMFs = emd(data[i,:], max_imf=5)
        
        feature_imfs=[]

            
        feature_imfs=compute_time_domain_features(IMFs[2,:])
        feature.extend(feature_imfs)


    return feature
        


        
        

In [11]:
# function for data augmentation

def train_augmentation(X,Y):

    total_samples=1152
    stride=192
    epoch_size=512
    # final_X=np.empty((0,64,epoch_size))
    # label=np.empty((0,1))
    print(f'with total_sample: {total_samples}, epoch size: {epoch_size} and strides: {stride}')
    final_data=np.empty((0,155))
    label=np.empty((0,1))
    
    for i in range(len(X)):
        #result=np.empty((4,64,512))
        with open(X[i], 'rb') as f:
            data = np.load(f)

            indices = np.arange(1, 17)
            indices = np.append(indices, np.arange(33, 49))
            indices = np.delete(indices, 9)

            # indices = np.arange(17, 32)
            # indices = np.append(indices, np.arange(49, 64))
            


            data = data[indices, :total_samples]

            # feature=feature_extraction(data)
            # feature=np.array(feature).reshape((1,-1))
            
            # if numeric_labels[type][Y[i]]==0:
            #     label=np.vstack((label, np.zeros((1,1))))
            # else:
            #     label=np.vstack((label, np.ones((1,1))))
            # final_data=np.vstack((final_data, feature))

            


            
            for j in range(0, total_samples, stride):
                if j+epoch_size >= total_samples:
                    break
            

                feature=feature_extraction(data)
                feature=np.array(feature).reshape((1,-1))
                # print(feature.shape)
                if numeric_labels[type][Y[i]]==0:
                    label=np.vstack((label, np.zeros((1,1))))
                else:
                    label=np.vstack((label, np.ones((1,1))))
                final_data=np.vstack((final_data, feature))       

        
            
    
    
        
    return final_data, label

In [12]:
# type="Long_words"
# subject_no=7
# X,Y=load_EEG(type, subject_no)
# data, label=train_augmentation(X,Y)

In [13]:
# data

In [14]:
def calculate_performance(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f'accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1 {f1}')
    return accuracy


In [15]:
def train_model(X_train, X_test, y_train, y_test):

  # Import other classifiers as needed

  # Train classifiers with different n_components values
  model_accuracies={}
  svm_rbf=SVC(kernel='rbf')

  svm_rbf.fit(X_train, y_train)
  y_pred_rbf=svm_rbf.predict(X_test)
  accuracy_rbf=calculate_performance(y_test, y_pred_rbf)
  model_accuracies['svm_rbf']=accuracy_rbf
  
  svm_linear = SVC(kernel='linear')
  svm_linear.fit(X_train, y_train)
  y_pred_linear = svm_linear.predict(X_test)
  print("pca linear performance: ")
  accuracy_linear=calculate_performance(y_test, y_pred_linear)
  model_accuracies['svm_linear']=accuracy_linear

  svm_poly = SVC(kernel='poly')
  svm_poly.fit(X_train, y_train)
  y_pred_poly = svm_poly.predict(X_test)
  print("pca linear performance: ")
  accuracy_poly=calculate_performance(y_test, y_pred_poly)
  model_accuracies['svm_poly']=accuracy_poly
  
  rfc = RandomForestClassifier()
  rfc.fit(X_train, y_train)
  y_pred_rfc = rfc.predict(X_test)
  print("Random Forest performance: ")
  accuracy_rfc=calculate_performance(y_test, y_pred_rfc)
  model_accuracies['rfc']=accuracy_rfc
 
  k = 5
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  y_pred_knn = knn.predict(X_test)
  print("KNN: ")
  accuracy_knn=calculate_performance(y_test, y_pred_knn)
  model_accuracies['knn']=accuracy_knn


  mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500,activation='relu', solver='adam', random_state=42)
  mlp.fit(X_train, y_train)
  y_pred = mlp.predict(X_test)
  y_pred_mlp = [round(value) for value in y_pred]
  print('MLP performance: ')
  accuracy_mlp=calculate_performance(y_test, y_pred_mlp)
  model_accuracies['mlp']=accuracy_mlp


  return model_accuracies

In [16]:
def get_data(type,subject_no):
    
    X,Y=load_EEG(type, subject_no)

    kfold=5
    skf = StratifiedKFold(n_splits=kfold, random_state=42, shuffle=True)

    average_accuracies={}
    
    for i, (train_index, test_index) in enumerate(skf.split(X, Y)):
        train_X=X[train_index]
        train_y=Y[train_index]
        test_X=X[test_index]
        test_y=Y[test_index]
        X_train, y_train  = train_augmentation(train_X, train_y)
        X_test, y_test = train_augmentation(test_X, test_y)
        y_train=y_train.reshape((-1))
        y_test=y_test.reshape((-1))
        # csp = CSP(n_components=4, reg=0.003, log=False, norm_trace=False)
        # csp.fit(X_train, y_train)
        # X_train_csp = csp.transform(X_train)
        # X_test_csp = csp.transform(X_test)

        scaler = StandardScaler()
        # train_data = scaler.fit_transform(X_train)
        # test_data = scaler.transform(X_test)

        # n_components =   100

        # pca = PCA(n_components=n_components)
        # X_train = pca.fit_transform(train_data)
        # X_test = pca.transform(test_data)

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)



        print(f'fold {i} performance: ')
        accuracies=train_model(X_train, X_test, y_train, y_test)
        for model_name, accuracy in accuracies.items():
            if model_name not in average_accuracies:
                average_accuracies[model_name] = []
            average_accuracies[model_name].append(accuracy)
    for model_name, accuracies in average_accuracies.items():
        average_accuracy = np.mean(accuracies)
        std_deviation = np.std(accuracies)
        print(f"{model_name} Average Accuracy: {average_accuracy}")
        print(f"{model_name} Standard Deviation: {std_deviation}")

        


    return 


In [17]:

def K_fold_train(X_train, X_test, y_train, y_test):


    svm_rbf=SVC(kernel='rbf')

    num_folds = 5
    kfold = KFold(n_splits=num_folds)
    scores_rbf = cross_val_score(svm_rbf, X_train, y_train, cv=kfold)
    for fold_idx, accuracy in enumerate(scores_rbf):
        print(f"Fold {fold_idx+1} Accuracy: {accuracy}")
    mean_accuracy_rbf = scores_rbf.mean()
    std_deviation_rbf = scores_rbf.std()
    print(f'mean_accuracy_rbf: {mean_accuracy_rbf}, std_deviation_rbf: {std_deviation_rbf}')
    
    svm_linear = SVC(kernel='linear')
    scores_linear = cross_val_score(svm_linear, X_train, y_train, cv=kfold)
    for fold_idx, accuracy in enumerate(scores_linear):
        print(f"Fold {fold_idx+1} Accuracy: {accuracy}")
    mean_accuracy_linear = scores_linear.mean()
    std_deviation_linear = scores_linear.std()
    print(f'mean_accuracy_rbf: {mean_accuracy_linear}, std_deviation_rbf: {std_deviation_linear}')


    
    rfc = RandomForestClassifier()
    scores_rfc = cross_val_score(rfc, X_train, y_train, cv=kfold)
    for fold_idx, accuracy in enumerate(scores_rfc):
        print(f"Fold {fold_idx+1} Accuracy: {accuracy}")
    mean_accuracy_rfc = scores_rfc.mean()
    std_deviation_rfc = scores_rfc.std()
    print(f'mean_accuracy_rbf: {mean_accuracy_rfc}, std_deviation_rbf: {std_deviation_rfc}')

    
    k = 5  
    knn = KNeighborsClassifier(n_neighbors=k)
    scores_knn = cross_val_score(knn, X_train, y_train, cv=kfold)
    for fold_idx, accuracy in enumerate(scores_knn):
        print(f"Fold {fold_idx+1} Accuracy: {accuracy}")
    mean_accuracy_knn = scores_knn.mean()
    std_deviation_knn = scores_knn.std()
    print(f'mean_accuracy_rbf: {mean_accuracy_knn}, std_deviation_rbf: {std_deviation_knn}')

    mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500,activation='relu', solver='adam', random_state=42)
    scores_mlp = cross_val_score(mlp, X_train, y_train, cv=kfold)
    for fold_idx, accuracy in enumerate(scores_mlp):
        print(f"Fold {fold_idx+1} Accuracy: {accuracy}")
    mean_accuracy_mlp = scores_mlp.mean()
    std_deviation_mlp = scores_mlp.std()
    print(f'mean_accuracy_rbf: {mean_accuracy_mlp}, std_deviation_rbf: {std_deviation_mlp}')



    svm_rbf=SVC(kernel='rbf')
    


    return 


In [18]:
# type="Long_words"
# subject_no=6
# get_data(type, subject_no)

In [19]:

# now = datetime.now()
# dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# with open('EMD_Attempts2.txt', 'a') as file:
#     sys.stdout = file
#     print("time: ", dt_string)
    
#     type="Long_words"
#     subject_no=2
#     print(f'{type}, {subject_no}')
#     print('features: mean, std, kurtosis, Energy, skewness, Median')
#     get_data(type, subject_no)

#     sys.stdout = sys.__stdout__

In [20]:

# now = datetime.now()
# dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# with open('EMD_Attempts2.txt', 'a') as file:
#     sys.stdout = file
#     print("time: ", dt_string)
#     print('**********************************************')
#     type="Short_Long_words"
#     subject_no=[1,5,8,9,10,14]
#     print(f'{type}')
#     print('features: mean, std, kurtosis, Energy, skewness, Median')
#     for i in range(len(subject_no)):
#         print(f'{subject_no[i]}')
#         get_data(type, subject_no[i])

#     sys.stdout = sys.__stdout__

In [21]:
type="Short_Long_words"
subject_no=14
get_data(type, subject_no)

with total_sample: 1152, epoch size: 512 and strides: 192


KeyboardInterrupt: 